In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from google.colab import drive
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

In [ ]:
# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define dataset paths
base_path = "/content/drive/My Drive/Dataset"
folders = ["Armyworms", "Cabbage Loopers"]
folder_paths = [os.path.join(base_path, folder) for folder in folders]

# Step 3: Initialize images and labels lists
images = []
labels = []

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 4: Function to load images
def load_images_from_folder(folder_path, label, images_list, labels_list):
    for filename in tqdm(os.listdir(folder_path), desc=f"Loading {os.path.basename(folder_path)}"):
        file_path = os.path.join(folder_path, filename)
        img = cv2.imread(file_path)

        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            img = cv2.resize(img, (224, 224))  # Resize to (224, 224)
            images_list.append(img)
            labels_list.append(label)

# Step 5: Load images with labels
for label, folder in enumerate(folder_paths):
    load_images_from_folder(folder, label, images, labels)

# Step 6: Convert lists to NumPy arrays and normalize
images = np.array(images, dtype=np.float32) / 255.0  # Normalize pixel values
labels = np.array(labels)
print(labels)

Loading Cabbage Loopers: 100%|██████████| 104/104 [00:01<00:00, 63.89it/s]


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
# Step 7: Shuffle
images, labels = shuffle(images, labels, random_state=42)

# Step 8: Confirm labels
print("Labels:", labels)

# Step 9: Split
X, Y = images, labels



Labels: [0 0 1 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 0 1
 1 0 0 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 1 1 0 1
 1 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1 0 0 1 1 0 1 1 0
 1 0 0 1 0 0 1 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 0 0 1 0
 0 1 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 1 1 1
 1 1 0 0 1 1 0 1 0 1 0 0 1 1]


In [ ]:
# Step 9: Prepare for K‑Fold
X, Y = images, labels
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, Y), start=1):
    print(f"\n=== Fold {fold}/{k} ===")
    X_train, X_val = X[train_idx], X[val_idx]
    Y_train, Y_val = Y[train_idx], Y[val_idx]

    # Data augmentation for this fold
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    train_gen = datagen.flow(X_train, Y_train, batch_size=32)

    # Load base model
    base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
    base_model.trainable = False  # Freeze base layers

    # Custom head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=output)

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train
    model.fit(train_gen,
              validation_data=(X_val, Y_val),
              epochs=20,
              callbacks=[
                  EarlyStopping(patience=5, restore_best_weights=True),
                  ReduceLROnPlateau(patience=3)
              ],
              verbose=1)

    # Evaluate
    _, acc = model.evaluate(X_val, Y_val, verbose=0)
    print(f"Fold {fold} accuracy: {acc:.4f}")
    fold_accuracies.append(acc)

# Final results
print("\nK‑Fold results:", fold_accuracies)
print("Average accuracy:", np.mean(fold_accuracies))



=== Fold 1/5 ===


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.4389 - loss: 0.8506 - val_accuracy: 0.5250 - val_loss: 0.7415 - learning_rate: 5.0000e-04
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.4742 - loss: 0.8088 - val_accuracy: 0.6000 - val_loss: 0.6750 - learning_rate: 5.0000e-04
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5831 - loss: 0.6821 - val_accuracy: 0.6750 - val_loss: 0.6356 - learning_rate: 5.0000e-04
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.6361 - loss: 0.6380 - val_accuracy: 0.7000 - val_loss: 0.6145 - learning_rate: 5.0000e-04
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6724 - loss: 0.5988 - val_accuracy: 0.7250 - val_loss: 0.5880 - learning_rate: 5.0000e-04
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.7431 - loss: 0.5163 - val_accuracy: 0.7250 - val_loss: 0.5671 - learning_rate: 5.0000e-04
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7124 - loss: 0.5615 - val_

In [ ]:
loss,acc = model.evaluate(X_val,Y_val, verbose=1)
print(f"Test accurancy: {acc:4f}")
print(f"Test loss: {loss:4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step - accuracy: 0.9175 - loss: 0.2064
Test accurancy: 0.923077
Test loss: 0.198063
